In [60]:
from googleapiclient.discovery import build

# Reading DEVELOPER_KEY from key.txt
def readKey(filename):
    key = None
    try:
        f = open(filename, "r")
        contents = f.read()
        if (contents == ''):
            raise
        else:
            key = contents
    except:
        print("Please paste your API key in '" + filename + "' file")
        f = open(filename, "w")
    f.close()
    return key

DEVELOPER_KEY = readKey('key.txt')
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

In [73]:
class YouTube:
    def __init__(self, KEY):
        self._YOUTUBE_API_SERVICE_NAME = 'youtube'
        self._YOUTUBE_API_VERSION = 'v3'
        self._KEY = KEY
        self.searchOptions = {
            'part': 'snippet',
            'fields': 'items(id,snippet(publishedAt,title,description))',
        }
        self.youtube = build(self._YOUTUBE_API_SERVICE_NAME, self._YOUTUBE_API_VERSION, 
                             developerKey=self._KEY)
        
        self.lastSearch = None
        self.results = []
    
    def search(self, q, limit=1, links=False, videoOnly=False):
        ''' 
        q - search query, (ex. "Whale Shark")
        limit - number of results (ex. 10 or 100)
        links - Generate YouTube video links for each search result. 
                   Can be found in video['url'] (ex. True, False) 
        videosOnly - Return only videos from the result (ex. True, False)
        '''
        
        searchResult = self.youtube.search().list(
            q=q,
            part=self.searchOptions['part'],
            fields=self.searchOptions['fields'],
            type='video' if videoOnly else 'video,channel,playlist',
            maxResults=limit
          ).execute()
        
        self.lastSearch = searchResult['items']
        
        # Generating YouTube video links
        if (links):
            for item in self.lastSearch:
                if (item['id']['kind'] == 'youtube#video'):
                    item['url'] = 'https://youtu.be/' + item['id']['videoId']
                elif (item['id']['kind'] == 'youtube#channel'):
                    item['url'] = 'https://www.youtube.com/channel/' + item['id']['channelId']
                
        # Appeding result to all previous search results
        self.results = self.lastSearch + self.results
                
        return self.lastSearch

yt = YouTube(DEVELOPER_KEY)

In [80]:
yt.search('TheBrainDit', limit=1, videoOnly=True)
yt.lastSearch

[{'id': {'kind': 'youtube#video', 'videoId': 'E4fr_2Z3rgw'},
  'snippet': {'publishedAt': '2019-06-24T07:00:00.000Z',
   'title': 'САМЫЕ ВКУСНЫЕ АМЕРИКАНСКИЕ СЛАДОСТИ? БРЕЙН И ДАША ПРОБУЮТ ВКУСНЯШКИ!',
   'description': 'Олег Брейн и Даша Рейн пробуют различные необычные вкусняшки и сладости из Америки! Плейлист с едой тут...'}}]

In [81]:
yt.results

[{'id': {'kind': 'youtube#video', 'videoId': 'E4fr_2Z3rgw'},
  'snippet': {'publishedAt': '2019-06-24T07:00:00.000Z',
   'title': 'САМЫЕ ВКУСНЫЕ АМЕРИКАНСКИЕ СЛАДОСТИ? БРЕЙН И ДАША ПРОБУЮТ ВКУСНЯШКИ!',
   'description': 'Олег Брейн и Даша Рейн пробуют различные необычные вкусняшки и сладости из Америки! Плейлист с едой тут...'}},
 {'id': {'kind': 'youtube#video', 'videoId': 'E4fr_2Z3rgw'},
  'snippet': {'publishedAt': '2019-06-24T07:00:00.000Z',
   'title': 'САМЫЕ ВКУСНЫЕ АМЕРИКАНСКИЕ СЛАДОСТИ? БРЕЙН И ДАША ПРОБУЮТ ВКУСНЯШКИ!',
   'description': 'Олег Брейн и Даша Рейн пробуют различные необычные вкусняшки и сладости из Америки! Плейлист с едой тут...'}},
 {'id': {'kind': 'youtube#video', 'videoId': 'mrGLBGXupHQ'},
  'snippet': {'publishedAt': '2019-06-24T06:00:05.000Z',
   'title': 'ЗЛАЯ МОНАШКА ВЕРНУЛАСЬ...',
   'description': 'Evil Nun: Ужас в школе - первый взгляд и обзор игры про зловещую монахиню! Что скрывает это злачное место? Давайт...'}},
 {'id': {'kind': 'youtube#video', 'vi